# High-performance and parallel computing for AI - Practical 9: Numba-CUDA and more GPU programming

IMPORTANT
=========

* CuPy behaves weirdly for me. Restart the kernel if you encounter weird errors.
* For these practicals we will be using a different `conda environment`. When opening a notebook or a terminal make sure you are using the **CuPy Kernel**!!!
* It's fine if you do not finish everything.

# IMPORTANT

Before you start (and before running any other GPU code on the servers) please run the following code, which limits the maximum GPU memory usage to $1.5$ GB and picks an L40s GPU and a Quadro GPU at random. **Please only run the code below once every time you restart the kernel!** 

In [1]:
import os

# CuPy-specific environment variables
os.environ["CUPY_GPU_MEMORY_LIMIT"] = "1573741824" # roughly 1.5 GB
os.environ["CUPY_ACCELERATORS"]="cutensor" # activates cutensor acceleration
os.environ["CUPY_TF32"] = "1" # activates tf32 tensor cores

## On goliat we have FIVE GPUs so here we pick two of those at random
## so that we do not overload the system.
## The way we do it is by figuring out the GPU UUIDs and then setting
## The CUDA_VISIBLE_DEVICES environment variable.
## Note: this is useful for other libraries as well (e.g., Jax, PyTorch, TF) in multi-GPU servers.

# To get these UUIDs you need to run nvidia-smi -q on the command line
quadro_UUIDs = ["GPU-4efa947b-abbd-7c6e-84f5-61241d34bb4b",
                "GPU-5eb524b0-2b1b-fe98-e6ed-b8fb5185e993"]

L40s_UUIDs = ["GPU-7bba1f33-03d2-016b-d42e-ced83c3ac243",
              "GPU-179d068a-3bea-91d7-1a8c-7017f55d6298",
              "GPU-ae634859-dd49-de46-9182-195639405eaa"]

from numpy.random import randint
# Picks an L40s and a Quadro GPU at random. The others will be invisible to CuPy
# NOTE: this only works if the environment variable is set BEFORE CuPy is first imported.
os.environ["CUDA_VISIBLE_DEVICES"] = L40s_UUIDs[randint(3)] + "," + quadro_UUIDs[randint(2)]

## CuPy and Numba will only see these GPUs and will assign them these device numbers:
L40sID = 0
quadro_ID = 1

## Tutorial 1 - Numba-CUDA

Numba-CUDA is a a spinoff of Numba which is now being developed separately. However, the numba-cuda docs are still currently part of the mainline numba docs. You can find them [here](https://numba.readthedocs.io/en/stable/cuda/index.html).

Numba-CUDA works similarly to CuPy JIT-Rawkernel. However, it is much better documented.

As a first example we show the solution to Question 3 of Practical 8 implemented using numba-CUDA (with a little help from CuPy). Please study this code as it will help you understand the basics of Numba-CUDA.

In [2]:
import cupy as cp
from numba import cuda
import numpy as np

from cupyx.profiler import benchmark

# NOTE: the following CUDA variables are available
# when writing Numba-CUDA kernels.
#
#     cuda.threadIdx.x, cuda.blockIdx.x, cuda.blockDim.x
#
# So that, for instance, you can get the thread ID
# and the stride in the x,y directions as follows:
#
#     tidx = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
#     ntidx = cuda.gridDim.x * cuda.blockDim.x
#     tidy = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
#     ntidy = cuda.gridDim.y * cuda.blockDim.y
#
# However, in Numba-CUDA this is made simpler thanks
# to the helper functions cuda.grid(dim) and cuda.gridsize(dim).
# Just do:
#
#     tidx, tidy = cuda.grid(2)
#     ntidx,ntidy = cuda.gridsize(2)

@cuda.jit
def myfun(x, y, z): # z is the OUTPUT. It will be overwritten!
    tidx, tidy = cuda.grid(2) 
    ntidx,ntidy = cuda.gridsize(2)
    for i in range(tidy, n, ntidy):
        for j in range(tidx, n, ntidx):
            z[i, j] = x[i,j]*y[i,j]

gridDim = (128, 128)
blockSize = (32, 32)

n = 4096 

# Initialising CuPy arrays directly on the device
acp = cp.random.randn(n, n, dtype=np.float32)
bcp = cp.random.randn(n, n, dtype=np.float32)
ccp = cp.empty((n, n), dtype=np.float32)

# While Numba-CUDA kernels are compatible with CuPy arrays
# in my experiment it is faster to convert CuPy arrays to
# Numba-CUDA arrays by hand. To do so, you simply call
# cuda.to_device(arr).
# The same command also works to move arrays stored on the host onto the device.
# However, doing this from CuPy avoids memory movement since they are already on the device.
a = cuda.to_device(acp)
b = cuda.to_device(bcp)
c = cuda.to_device(ccp)

# You can now call the kernel like this. c will be overwritten
myfun[gridDim, blockSize](a, b, c)

# Getting the array back into CuPy
# Again, CuPy is smart and no copy/memory movement
# will be made
ccp = cp.asarray(c)

# This is not needed here, but this is how you get
# a numba array back to the host.
ch = c.copy_to_host()

assert (ccp == acp*bcp).all()

# You can wrap everything into a function 
# that only works with CuPy arrays for convenience
# This is also so that we can benchmark it
def mywrappedfun(gd, bs, cp_inputs):
    nc_inputs = (cuda.to_device(item) for item in cp_inputs)
    myfun[gd,bs](*nc_inputs)

ccp = cp.zeros((n, n), dtype=np.float32)
mywrappedfun(gridDim, blockSize, (acp, bcp, ccp))

# Just to make sure c actually gets overwritten.
# This also proves that using cuda.to_device does not move any memory
assert ccp.sum() != 0 

print(benchmark(mywrappedfun, (gridDim, blockSize, (a,b,c)), n_repeat=100))

mywrappedfun        :    CPU:    62.316 us   +/-  4.039 (min:    58.040 / max:    79.410) us     GPU-0:   344.753 us   +/-  3.965 (min:   339.968 / max:   361.472) us


Note that the above is as fast as the CuPy JIT-Rawkernel.
However, the Numba-CUDA documentation is more extensive so I hope that it gives the developer more options. Both Numba-CUDA and CuPy are in active development so it is hard to tell how these functionalities will evolve in the future. It may be useful to know both, and Numba-CUDA shares some similarities with actual CUDA so it is good for teaching.

Before we proceed, a quick timing of the cost of memory movement:

In [3]:
# From Numba-CUDA to CuPy
print(benchmark(cp.asarray, (c,), n_repeat=1000))

# From CuPy to Numba-CUDA
print(benchmark(cuda.to_device, (ccp,), n_repeat=1000))

# From Numba-CUDA to Host
print(benchmark(lambda x : x.copy_to_host(), (c,), n_repeat=100))

# From CuPy to host
print(benchmark(cp.asnumpy, (ccp,), n_repeat=100))

asarray             :    CPU:     9.890 us   +/-  1.817 (min:     9.088 / max:    50.526) us     GPU-0:    12.725 us   +/-  1.923 (min:    11.264 / max:    53.248) us
to_device           :    CPU:    44.798 us   +/-  9.732 (min:    41.469 / max:   249.523) us     GPU-0:    48.653 us   +/-  9.826 (min:    44.096 / max:   253.952) us
<lambda>            :    CPU: 10812.485 us   +/- 782.521 (min:  9434.302 / max: 12527.588) us     GPU-0: 10819.399 us   +/- 782.586 (min:  9441.248 / max: 12535.136) us
asnumpy             :    CPU: 11146.435 us   +/- 742.569 (min:  9433.901 / max: 14843.432) us     GPU-0: 11152.253 us   +/- 742.669 (min:  9439.552 / max: 14849.664) us


As you can see, moving memory between host and device is much more expensive.

## Question 1 - Local reductions and dynamic shared memory

Consider the following CUDA snippet that implements a local reduction operation:

```C++
__global__ void reduction(float *g_odata, float *g_idata)
{
    // dynamically allocated shared memory
    extern  __shared__  float temp[];

    int tid = threadIdx.x;

    // first, each thread loads data into shared memory
    temp[tid] = g_idata[tid];

    // next, we perform binary tree reduction
    for (int d=blockDim.x/2; d>0; d=d/2) {
      __syncthreads();  // ensure previous step completed 
      if (tid<d)
          temp[tid] += temp[tid+d];
    }

    // finally, first thread puts result into global memory
    if (tid == 0)
        g_odata[0] = temp[0];
}
```

Note that this uses syncthreads and dynamic shared memory. First, read the code above and understand what it does (it computes a sum of all entries in `g_idata` and stores the output in `g_odata`).

* Why was syncthreads used?
* This looks like a convoluted approach to perform a sum. Can you think about why something like this would be needed?

By looking at the above tutorial and at the lecture slides convert this code into Python by using Numba-CUDA and call it by inserting it into the Python script provided below, which you will also have to modify (follow the code comments).

**Hint:** Note that this example uses dynamic shared memory! The [Numba docs](https://numba.readthedocs.io/en/stable/cuda/memory.html) and the course slides may be helpful.

**Hint:** In Python you will have to use the following while loop in place of a foor loop:
```python
d = cuda.blockDim.x//2 # You need integer division here
while d > 0:
    # loop body here
    d = d//2
```

In [4]:
%%script true
# NOTE: remove the line above else the cell won't run

import cupy as cp
from numba import cuda
import numpy as np

from cupyx.profiler import benchmark

num_blocks = 1
num_threads = 512
num_elements = num_blocks*num_threads

# NOTE: Allocated on the host
h_idata = 10.*np.random.rand(num_elements)
h_idata = h_idata.astype(np.float32) # input data
ex_sum = h_idata.sum() # exact sum computed by host

d_idata = None # FIX THIS. Move h_idata onto the device
d_odata = None # Fix this. Initialise an empty array with a single float32 entry on the device to hold the output

# Modify this and JIT-it with Numba
def reduction(g_odata, g_idata):
    raise NotImplementedError

shared_memory_size = None # MODIFY THIS. How big should this be? Remember: here you need memory in bytes, not array entries!!!
reduction[FIXME](d_odata, d_idata) # MODIFY THIS by replacing FIXME with the correct kernel parameters.

computed_sum = None # Get d_odata back to the host. Its first entry is the computed sum.

print("Reduction error: %.3e" % abs(computed_sum[0] - ex_sum))

## Solution to question 1

In [11]:
import cupy as cp
from numba import cuda
import numpy as np

from cupyx.profiler import benchmark

num_blocks = 1
num_threads = 512
num_elements = num_blocks*num_threads

# NOTE: Allocated on the host
h_idata = 10.*np.random.rand(num_elements)
h_idata = h_idata.astype(np.float32) # input data
ex_sum = h_idata.sum() # exact sum computed by host

d_idata = cuda.to_device(h_idata) # FIX THIS. Move h_idata onto the device
cp_odata = cp.empty((1,), dtype=np.float32)
d_odata = cuda.to_device(cp_odata) # Fix this. Initialise an empty array with a single float32 entry on the device to hold the output

# Modify this and JIT-it with Numba
@cuda.jit
def reduction(g_odata, g_idata):
    temp = cuda.shared.array(0, dtype=np.float32)

    tid = cuda.threadIdx.x

    # first, each thread loads data into shared memory
    temp[tid] = g_idata[tid]

    # next, we perform binary tree reduction
    d = cuda.blockDim.x//2 # You need integer division here
    while d > 0:
        cuda.syncthreads() # ensure previous step completed
        if tid < d:
            temp[tid] += temp[tid+d]
            
        d = d//2

    # finally, first thread puts result into global memory
    if tid == 0:
        g_odata[0] = temp[0]
        
shared_memory_size = np.dtype(np.float32).itemsize*num_threads # MODIFY THIS. How big should this be?
reduction[num_blocks, num_threads, 0, shared_memory_size](d_odata, d_idata) # MODIFY THIS by replacing FIXME with the correct kernel parameters.

computed_sum = d_odata.copy_to_host() # Get d_odata back to the host. Its first entry is the computed sum.

print("Reduction error: %.3e" % abs(computed_sum[0] - ex_sum))

Reduction error: 0.000e+00


/opt/tljh/user/envs/cupy/lib/python3.12/site-packages/numba_cuda/numba/cuda/dispatcher.py:663: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


## Question 2

The code from Question 1 currently assumes the number of threads is a power of 2.
Extend it to handle the general case by finding the largest power of 2 less than
`blockSize`, and adding the elements beyond that point to the corresponding
first set of elements of that size. Test it with 192 threads

## Solution to Question 2

## Question 3 - Constant memory, static shared memory, device functions

This question is a bit silly, but is a good way to try different things. Modify the code from Question 1 in three ways:

1- Using static memory.

2- Saving the number `2` used in the loop as a constant variable using `cuda.const.array_like`.

3- Defining a [device function](https://numba.readthedocs.io/en/stable/cuda/device-functions.html) that computes $d=d//2$.

## Solution to question 3

## Question 4

The code from Question 1 currently performs the reduction operation for a single thread block.
Modify the code to perform reduction using multiple blocks (say $4$ of them) with each block working with a different section of the input array.

There are two ways in which the partial sums from each block can be summed:
* Each block puts its partial sum into a different element of the output
array, and then these are transferred to the host and summed there;
* An atomic addition is used to safely increment a single global sum.

Implement both and check that you get the correct answer.

Finally, implement the reduction using:
* The `cuda.reduce` decorator.
* `cupy.sum`.

Time all four versions. Which one is faster? Can you see why it is much easier to not write reductions yourself?

## Solution to question 4